transaction matrix 만들기

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm

from tools_ryu import mytool
from multi_run_v3.data_pp import DataPreprocess
from multi_run_v3.momentum import make_group_mask

In [2]:
data = pd.read_pickle("Data/cmktcap_data_final.pickle")

data_pp = DataPreprocess()
data_pp.initialize(data=data)
data_pp.make_mask(mktcap_thresh=1_000_000,
                  vol_thresh=1_000_000,
                  ma=True)

In [15]:
result = []
for day in ['MON','TUE','WED','THU','FRI','SAT','SUN']:
    mask_dict = make_group_mask(
                            price_df=data_pp.price_df,
                            weekly_rtn_df=data_pp.weekly_rtn_df,
                            mask_df=data_pp.mask,
                            n_group=10,
                            coin_group=10,
                            day_of_week=day
                            )
    
    df = mask_dict['Q1'].fillna(0) * 1 + mask_dict['Q2'].fillna(0) * 2 + mask_dict['Q3'].fillna(0) * 3 + mask_dict['Q4'].fillna(0) * 4 + mask_dict['Q5'].fillna(0) * 5 \
        + mask_dict['Q6'].fillna(0) * 6 + mask_dict['Q7'].fillna(0) * 7 + mask_dict['Q8'].fillna(0) * 8 + mask_dict['Q9'].fillna(0) * 9 + mask_dict['Q10'].fillna(0) * 10
    df = df.replace(0, np.nan).dropna(axis=1, thresh=1).replace(np.nan, 0) 
    
    # Then we initialize our transition count matrix
    transition_counts = pd.DataFrame(0, index=[0,1,2,3,4,5,6,7,8,9,10], columns=[0,1,2,3,4,5,6,7,8,9,10])
    
    # We then calculate the transition counts
    for coin in tqdm(df.columns):  
        for i in range(1, len(df)):
            prev_group = df[coin].iloc[i-1]
            current_group = df[coin].iloc[i]
            transition_counts.at[prev_group, current_group] += 1
    
    # Finally, we convert these counts into probabilities to get our transition matrix
    transition_counts.replace(0, np.nan, inplace=True)
    #transition_matrix = transition_counts.div(transition_counts.sum(axis=1), axis=0)
    result.append(transition_counts)

100%|██████████| 1512/1512 [00:20<00:00, 73.17it/s]


In [16]:
# i-th row and j-th column represents the probability of a coin moving from group i to group j from one day to the next
result_df = pd.DataFrame(0,
                         index=[0,1,2,3,4,5,6,7,8,9,10],
                         columns=[0,1,2,3,4,5,6,7,8,9,10])
for d in result:
    result_df += d
    
result_df = result_df / 7

In [17]:
transition_matrix = result_df.iloc[1:,1:].div(result_df.iloc[1:,1:].sum(axis=1), axis=0)
transition_matrix

,1,2,3,4,5,6,7,8,9,10
1,0.176904,0.113876,0.091367,0.078362,0.078391,0.075020,0.075425,0.082861,0.094708,0.133086
2,0.089264,0.103645,0.105279,0.101021,0.101451,0.101623,0.099845,0.101867,0.100963,0.095042
3,0.065153,0.089214,0.098771,0.107713,0.111122,0.114317,0.114973,0.109953,0.102793,0.085991
4,0.054174,0.081253,0.099107,0.110295,0.118582,0.120728,0.120288,0.112867,0.100401,0.082305
5,0.050587,0.078102,0.099472,0.113236,0.118231,0.123411,0.121084,0.112158,0.100508,0.083210
6,0.053381,0.081718,0.100586,0.111163,0.119625,0.118588,0.117183,0.116118,0.100104,0.081534
7,0.057957,0.088864,0.106047,0.112597,0.111547,0.111774,0.111278,0.109265,0.101708,0.088963
8,0.071060,0.101990,0.109701,0.109971,0.103068,0.102869,0.099850,0.099240,0.100048,0.102203
9,0.104291,0.123614,0.106344,0.096660,0.089993,0.086355,0.086850,0.088917,0.098218,0.118759
10,0.234793,0.129958,0.085790,0.067186,0.057475,0.053287,0.057777,0.065257,0.085790,0.162687
